In [1]:
import math
import re
import torch
from torch import nn
from torch.nn import functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
import random
seed_value = 42

# 设置随机种子
torch.manual_seed(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)

# 如果使用GPU，还可以设置相关的随机种子
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
data = pd.read_csv('基准数据集.csv')
sequences = data.Sequence
labels = data.Label.values
data

,Meta,Sequence,Label
0,lcl|Athaliana_AT5G45455.1,GAAATTCTTTGGAGCTCAGTGGCCCAACAAGATTATAATCCGAAAA...,1
1,lcl|Athaliana_AT2G44925.1,AAGGAAAAAAAAAAAGGAATTTCGTTTCTCTTGGTGTTAAAAGGAG...,1
2,lcl|Athaliana_AT3G51660.1,GTCAGATTTGAAACTCAAAGATATTATTTCAAAAATTGTTCACATC...,1
3,lcl|Athaliana_AT4G06655.1,ATGAACTTGCCTTTCGATGACAAAATCAAACTCAGATTCCGACTAC...,1
4,URS0000291A56_3702,GGCATCCCGTCCTTAATTGGTCC,1
...,...,...,...
15995,VIT_201s0010g02000.1,ATGGGCTCAATAGCAGGGAATTATGGTGCATGCATTTTTGTGGCAG...,0
15996,VIT_208s0007g01060.2,ATGGATTCCTCTCGCGAGTTCGTCAAGGACGTCAAGCGTGTCATTG...,0
15997,VIT_214s0060g01630.2,ATGGCAACTTTTGCCAAACCAGAGAATGCTTTGAAGCGAGCTGAAG...,0
15998,VIT_208s0007g07570.2,ATGGGGGCGAGTCGAAAACTACAAGGCGAGATTGACAGGGTTCTGA...,0


In [4]:
def extract_sequences_from_fasta(file_path):
    headers = []
    sequences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        current_header = ''
        current_sequence = ''
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                if current_header and current_sequence:
                    headers.append(current_header)
                    sequences.append(current_sequence)

                current_header = line[1:]
                current_sequence = ''
            else:
                current_sequence += line

        # 处理最后一个序列
        if current_header and current_sequence:
            headers.append(current_header)
            sequences.append(current_sequence)

    data = {'Meta': headers, 'Sequence': sequences}
    df = pd.DataFrame(data)
    return df

# 读取FASTA文件并提取序列
file_path = r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\cRNA\Zea mays.fasta'
datah = extract_sequences_from_fasta(file_path)
print(datah.shape[0])

file_path = r'C:\Users\26970\Desktop\package\DL\PINC-main\DataSets\TestSets\\ncRNA\Zea mays.fasta'
datah_nc = extract_sequences_from_fasta(file_path)
print(datah_nc.shape[0])
datah = pd.concat([datah,datah_nc]).reset_index(drop=True)

pattern = r'(?:^|\s)([^ ]+)'  # 匹配第一个空格前的非空格字符序列作为标识部分
datah['Sequence'] = [re.search(pattern, identifier).group(1) for identifier in datah['Sequence']]
label_data = [0] * 7406 + [1] * 7406

y = pd.DataFrame(label_data, columns=['Label'])
datah.reset_index(drop=True, inplace=True)
datah = datah.join(y)

sequences_te = datah.Sequence
labels_te = datah.Label.values

datah

7406
7406


,Meta,Sequence,Label
0,Zm00001d033646_T002,ATGGCCGCCGCGACAGAGAAGACGGCTGAGGACATCCGCCGCGAGC...,0
1,Zm00001d002782_T001,ATGGCCTCTCCTTCCCCTTCTTCCCCCGCCCGCGCCTCCGGCCGCC...,0
2,Zm00001d007357_T004,ATGTTTCTTAAGGAACTAGACTTACAAAAGAGTTGTGTGAAACATC...,0
3,Zm00001d008727_T002,ATGGCTCAGATCTTGCTCCACGGCACGCTCCACGCCACCATCTTCG...,0
4,Zm00001d003476_T001,ATGCACCAGGATGCACACGAGTTCTTAAATTTTCTTTTGAATGAAC...,0
...,...,...,...
14807,URS0001BC9443_4577 Zea mays mir-393 microRNA p...,CUCCAAAGGGAUCGCAUUGAUCUAACCUGCCGAUCGACGCCGACGU...,1
14808,URS00021F59AE_4577 Zea mays (maize) zm00001e01...,GGGGAAAGTCTTTGACTTGCTAGCCAGCCGTAGTAGTAATAATACG...,1
14809,URS00021F600D_4577 Zea mays (maize) zm00001e03...,ACTTACCTGGACGGGGTCGACGGGCTATCAAGAAGGCCCGTGGCCT...,1
14810,URS00021F6599_4577 Zea mays (maize) zm00001e04...,TCTTGCCTTGCGCGATCTAAGTCTAAGGAAATACAAAAATGCGCTT...,1


In [5]:
pat = re.compile('[AGCTagct]')

def pre_process(text):
    text = pat.findall(text)
    text = [each.lower() for each in text]
    return text

x = sequences.apply(pre_process)

In [6]:
x_te = sequences_te.apply(pre_process)
x_te

0        [a, t, g, g, c, c, g, c, c, g, c, g, a, c, a, ...
1        [a, t, g, g, c, c, t, c, t, c, c, t, t, c, c, ...
2        [a, t, g, t, t, t, c, t, t, a, a, g, g, a, a, ...
3        [a, t, g, g, c, t, c, a, g, a, t, c, t, t, g, ...
4        [a, t, g, c, a, c, c, a, g, g, a, t, g, c, a, ...
                               ...                        
14807    [c, c, c, a, a, a, g, g, g, a, c, g, c, a, g, ...
14808    [g, g, g, g, a, a, a, g, t, c, t, t, t, g, a, ...
14809    [a, c, t, t, a, c, c, t, g, g, a, c, g, g, g, ...
14810    [t, c, t, t, g, c, c, t, t, g, c, g, c, g, a, ...
14811    [a, c, c, t, g, g, a, c, g, t, a, g, c, g, a, ...
Name: Sequence, Length: 14812, dtype: object

In [7]:
word_set = set()

for lst in x:
    for word in lst:
        word_set.add(word)

word_list = list(word_set)
word_index = dict([(each, word_list.index(each) + 1) for each in word_list])
word_index

{'a': 1, 'c': 2, 't': 3, 'g': 4}

In [8]:
word_set_te = set()

for lst_te in x_te:
    for word_te in lst_te:
        word_set_te.add(word_te)
        
word_list_te = list(word_set_te)
word_index_te = dict([(each_te, word_list_te.index(each_te) + 1) for each_te in word_list_te])
word_index_te

{'a': 1, 'c': 2, 't': 3, 'g': 4}

In [9]:
text = x.apply(lambda x: [word_index.get(word, 0) for word in x])

In [10]:
text_te = x_te.apply(lambda x_te: [word_index_te.get(word, 0) for word in x_te])
text_te

0        [1, 3, 4, 4, 2, 2, 4, 2, 2, 4, 2, 4, 1, 2, 1, ...
1        [1, 3, 4, 4, 2, 2, 3, 2, 3, 2, 2, 3, 3, 2, 2, ...
2        [1, 3, 4, 3, 3, 3, 2, 3, 3, 1, 1, 4, 4, 1, 1, ...
3        [1, 3, 4, 4, 2, 3, 2, 1, 4, 1, 3, 2, 3, 3, 4, ...
4        [1, 3, 4, 2, 1, 2, 2, 1, 4, 4, 1, 3, 4, 2, 1, ...
                               ...                        
14807    [2, 2, 2, 1, 1, 1, 4, 4, 4, 1, 2, 4, 2, 1, 4, ...
14808    [4, 4, 4, 4, 1, 1, 1, 4, 3, 2, 3, 3, 3, 4, 1, ...
14809    [1, 2, 3, 3, 1, 2, 2, 3, 4, 4, 1, 2, 4, 4, 4, ...
14810    [3, 2, 3, 3, 4, 2, 2, 3, 3, 4, 2, 4, 2, 4, 1, ...
14811    [1, 2, 2, 3, 4, 4, 1, 2, 4, 3, 1, 4, 2, 4, 1, ...
Name: Sequence, Length: 14812, dtype: object

In [11]:
text_len = 1000

pad_text = [l + (text_len - len(l)) * [0] if len(l) < text_len else l[:text_len] for l in text]

pad_text = np.array(pad_text)

In [12]:
text_len = 1000

pad_text_te = [l + (text_len - len(l)) * [0] if len(l) < text_len else l[:text_len] for l in text_te]

pad_text_te = np.array(pad_text_te)
pad_text_te

array([[1, 3, 4, ..., 2, 3, 4],
       [1, 3, 4, ..., 0, 0, 0],
       [1, 3, 4, ..., 1, 4, 4],
       ...,
       [1, 2, 3, ..., 0, 0, 0],
       [3, 2, 3, ..., 0, 0, 0],
       [1, 2, 2, ..., 3, 3, 2]])

In [13]:
x_train, x_test, y_train, y_test = train_test_split(pad_text, labels, test_size=0.3)
# x_train = pad_text
# x_test = pad_text_te
# y_train = labels
# y_test = labels_te

In [14]:
class Mydataset(torch.utils.data.Dataset):
    def __init__(self, text_list, label_list):
        self.text_list = text_list
        self.label_list = label_list
    
    def __getitem__(self,index):
        text = torch.LongTensor(self.text_list[index])
        label = self.label_list[index]
        return text, label
    
    def __len__(self):
        return len(self.text_list)

train_ds = Mydataset(x_train, y_train)
test_ds = Mydataset(x_test, y_test)

batch_size = 32

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=False)

In [15]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."

    def __init__(self, d_model, dropout=0.1, max_len=text_len) :
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len,dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0,1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [16]:
# embed_dim = 2 ** (int(np.log2(len(word_list) ** 0.25)) + 2)  # 经验值

embedding_dim = 50
print('embedding_dim:', embedding_dim)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.em = nn.Embedding(len(word_list) + 1, embedding_dim)  # 对0也需要编码
        self.pos = PositionalEncoding(embedding_dim)
        self.encoder_layer = nn.TransformerEncoderLayer(embedding_dim, nhead=5)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=3)
        self.fc1 = nn.Linear(embedding_dim, 128)
        self.fc2 = nn.Linear(128, 2)
           
    def forward(self, inputs):
        x = self.em(inputs)
        x = self.pos(x)
        x = x.float()
        x = self.transformer_encoder(x)
        x = torch.sum(x, dim=0)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

embedding_dim: 50


In [17]:
model = Net()
model = model.to('cuda')

loss = nn.CrossEntropyLoss()
loss = loss.to('cuda')

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [18]:
def fit(model, optimizer, train_dl, test_dl):
    
    tr_correct = 0  # 预测正确的个数
    tr_total = 0  # 总样本数
    tr_loss = 0
    tr_TP = 0
    tr_TN = 0
    tr_FP = 0
    tr_FN = 0
    
    model.train()  # 训练模式
    for x, y in train_dl:
        x = x.permute(1, 0)
        x, y = x.to('cuda'), y.to('cuda')
        y_pred = model(x)
        loss_value = loss(y_pred, y)
        #flood=(loss_value - 0.002).abs() + 0.002  # 洪泛函数：防止过拟合
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
        
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)  # 在预测结果 y_pred 的每个样本上找到具有最大值的索引
            tr_correct += (y_pred == y).sum().item()
            tr_TP += ((y_pred == y) & (y == 1)).sum().item()
            tr_FN += ((y_pred != y) & (y == 1)).sum().item()
            tr_FP += ((y_pred != y) & (y == 0)).sum().item()
            tr_TN += ((y_pred == y) & (y == 0)).sum().item()
            tr_total += len(y)
            tr_loss += loss_value.item()  # 最后的loss还要除以batch数
            
    """1个epoch训练结束后，计算训练集的各个指标"""
    epoch_tr_loss = tr_loss / len(train_dl)
    epoch_tr_accuracy = tr_correct / tr_total
    epoch_tr_MCC = (tr_TP * tr_TN - tr_TP * tr_FN) / (math.sqrt((tr_TP + tr_FP) * (tr_TP + tr_FN) * (tr_TN + tr_FP) * (tr_TN + tr_FN)+ 0.01)) 
    epoch_tr_SE=tr_TP/(tr_TP+tr_FN+ 0.01) 
    epoch_tr_SPC = tr_TN / (tr_TN + tr_FP+ 0.01) 
    epoch_tr_PPV= tr_TP / (tr_TP + tr_FP+ 0.01) 
    epoch_tr_NPV= tr_TN / (tr_TN + tr_FN+ 0.01) 
    epoch_tr_recall = tr_TP / (tr_TP + tr_FN+ 0.01) 
    epoch_tr_precision = tr_TP / (tr_TP + tr_FP+ 0.01) 
    epoch_tr_F1 = (2 * epoch_tr_precision * epoch_tr_recall) / (epoch_tr_precision + epoch_tr_recall+ 0.01) 
    
    te_correct = 0  # 预测正确的个数
    te_total = 0  # 总样本数
    te_loss = 0
    te_TP = 0
    te_TN = 0
    te_FP = 0
    te_FN = 0
    
    model.eval()  # 评估模式
    with torch.no_grad():# 是一个上下文管理器，用于在块中禁用梯度计算，以减少内存使用并加快计算速度
        for x, y in test_dl:
            x = x.permute(1, 0)
            x, y = x.to('cuda'), y.to('cuda')
            y_pred = model(x)
            loss_value = loss(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            te_correct += (y_pred == y).sum().item()
            te_TP += ((y_pred == y) & (y == 1)).sum().item()
            te_FN += ((y_pred != y) & (y == 1)).sum().item()
            te_FP += ((y_pred != y) & (y == 0)).sum().item()
            te_TN += ((y_pred == y) & (y == 0)).sum().item()
            te_total += len(y)
            te_loss += loss_value.item()
        
    """1个epoch训练结束后，计算测试集的各个指标"""
    epoch_te_loss = te_loss / len(test_dl)
    epoch_te_accuracy = te_correct / te_total
    epoch_te_MCC = (te_TP * te_TN - te_TP * te_FN) / (math.sqrt((te_TP + te_FP) * (te_TP + te_FN) * (te_TN + te_FP) * (te_TN + te_FN)+ 0.01)) 
    epoch_te_SE=te_TP/(te_TP+te_FN  + 0.01)
    epoch_te_SPC = te_TN / (te_TN + te_FP + 0.01) 
    epoch_te_PPV= te_TP / (te_TP + te_FP + 0.01) 
    epoch_te_NPV= te_TN / (te_TN + te_FN + 0.01)
    epoch_te_recall = te_TP / (te_TP + te_FN + 0.01) 
    epoch_te_precision = te_TP / (te_TP + te_FP + 0.01)
    epoch_te_F1 = (2 * epoch_te_precision * epoch_te_recall) / (epoch_te_precision + epoch_te_recall + 0.01)

    return epoch_tr_loss, epoch_tr_accuracy, epoch_tr_MCC, epoch_tr_SE,epoch_tr_F1, epoch_tr_SPC,epoch_tr_PPV, epoch_tr_NPV,epoch_te_loss, epoch_te_accuracy, epoch_te_MCC, epoch_te_SE, epoch_te_SPC,epoch_te_PPV,epoch_te_F1,epoch_te_NPV
#     return epoch_tr_loss, epoch_tr_accuracy,epoch_te_loss, epoch_te_accuracy

In [19]:
tr_loss = []
tr_accuracy = []
tr_MCC = []
tr_SE = []
tr_SPC = []
tr_PPV=[]
tr_NPV=[]
tr_AUC=[]
tr_F1=[]

te_loss = []
te_accuracy = []
te_MCC = []
te_SE = []
te_SPC = []
te_PPV=[]
te_NPV=[]
te_AUC=[]
te_F1=[]
from sklearn import metrics

In [20]:
epochs = 80

for epoch in range(epochs): 
    print(f'{epoch} : ',end = '')
    epoch_tr_loss, epoch_tr_accuracy, epoch_tr_MCC, epoch_tr_SE,epoch_tr_F1, epoch_tr_SPC,epoch_tr_PPV, epoch_tr_NPV,epoch_te_loss, epoch_te_accuracy, epoch_te_MCC, epoch_te_SE, epoch_te_SPC,epoch_te_PPV,epoch_te_F1,epoch_te_NPV= fit(model, optimizer, train_dl, test_dl)
    
    tr_loss.append(epoch_tr_loss)
    tr_accuracy.append(epoch_tr_accuracy)
    tr_MCC.append(epoch_tr_MCC)
    tr_SE.append(epoch_tr_SE)
    tr_SPC.append(epoch_tr_SPC)
    tr_PPV.append(epoch_tr_PPV)
    tr_NPV.append(epoch_tr_NPV)
    tr_F1.append(epoch_tr_F1)
    "tr_AUC.append(epoch_tr_AUC)"
    
    te_loss.append(epoch_te_loss)
    te_accuracy.append(epoch_te_accuracy)
    print(epoch_te_accuracy)
    te_MCC.append(epoch_te_MCC)
    te_SE.append(epoch_te_SE)
    te_SPC.append(epoch_te_SPC)
    te_PPV.append(epoch_te_PPV)
    te_NPV.append(epoch_te_NPV)
    te_F1.append(epoch_te_F1)
    "te_AUC.append(epoch_te_AUC)"

0 : 0.8085416666666667
1 : 0.575
2 : 0.8110416666666667
3 : 0.6247916666666666
4 : 0.8291666666666667
5 : 0.7725
6 : 0.8197916666666667
7 : 0.8122916666666666
8 : 0.8352083333333333
9 : 0.7077083333333334
10 : 0.783125
11 : 0.7779166666666667
12 : 0.77875
13 : 0.8610416666666667
14 : 0.8352083333333333
15 : 0.7439583333333334
16 : 0.7770833333333333
17 : 0.8535416666666666
18 : 0.8297916666666667
19 : 0.506875
20 : 0.6233333333333333
21 : 0.84875
22 : 0.8660416666666667
23 : 0.5541666666666667
24 : 0.700625
25 : 0.671875
26 : 0.875625
27 : 0.873125
28 : 0.8597916666666666
29 : 0.8808333333333334
30 : 0.876875
31 : 0.8825
32 : 0.8877083333333333
33 : 0.8377083333333334
34 : 0.8604166666666667
35 : 0.874375
36 : 0.8864583333333333
37 : 0.8777083333333333
38 : 0.9029166666666667
39 : 0.8954166666666666
40 : 0.8979166666666667
41 : 0.8864583333333333
42 : 0.8722916666666667
43 : 0.9075
44 : 0.8966666666666666
45 : 0.8729166666666667
46 : 

KeyboardInterrupt: 

In [ ]:
column_name = ['loss', 'accuracy', 'MCC', 'SE', 'SPC', 'PPV', 'NPV','F1']

tr_loss = pd.Series(tr_loss,dtype='float64')
tr_accuracy = pd.Series(tr_accuracy,dtype='float64')
tr_MCC = pd.Series(tr_MCC,dtype='float64')
tr_SE = pd.Series(tr_SE,dtype='float64')
tr_SPC = pd.Series(tr_SPC,dtype='float64')
tr_PPV = pd.Series(tr_PPV,dtype='float64')
tr_NPV = pd.Series(tr_NPV,dtype='float64')
tr_F1 = pd.Series(tr_F1,dtype='float64')

tr_result = pd.concat([tr_loss, tr_accuracy, tr_MCC, tr_SE, tr_SPC, tr_PPV, tr_NPV,tr_F1], axis=1)
tr_result.columns = column_name

In [ ]:
column_name = ['loss', 'accuracy', 'MCC', 'SE', 'SPC', 'PPV', 'NPV','F1']

te_loss = pd.Series(te_loss)
te_accuracy = pd.Series(te_accuracy)
te_MCC = pd.Series(te_MCC)
te_SE = pd.Series(te_SE)
te_SPC = pd.Series(te_SPC)
te_PPV = pd.Series(te_PPV)
te_NPV = pd.Series(te_NPV)
te_F1= pd.Series(te_F1)

te_result = pd.concat([te_loss, te_accuracy, te_MCC, te_SE, te_SPC, te_PPV, te_NPV,te_F1], axis=1)
te_result.columns = column_name
#te_result.index = [*range(1, epochs + 1)]

In [ ]:
max(te_result.accuracy)

In [23]:
# save
tr_result.to_csv(r'./调参前后/Word_embedding_TF_train.csv')
te_result.to_csv(r'./调参前后/Word_embedding_TF_valid.csv')

In [ ]:
epochs = 80

for epoch in range(epochs): 
    print(f'{epoch} : ',end = '')
    epoch_tr_loss, epoch_tr_accuracy, epoch_tr_MCC, epoch_tr_SE,epoch_tr_F1, epoch_tr_SPC,epoch_tr_PPV, epoch_tr_NPV,epoch_te_loss, epoch_te_accuracy, epoch_te_MCC, epoch_te_SE, epoch_te_SPC,epoch_te_PPV,epoch_te_F1,epoch_te_NPV= fit(model, optimizer, train_dl, test_dl)
    
    tr_loss.append(epoch_tr_loss)
    tr_accuracy.append(epoch_tr_accuracy)
    tr_MCC.append(epoch_tr_MCC)
    tr_SE.append(epoch_tr_SE)
    tr_SPC.append(epoch_tr_SPC)
    tr_PPV.append(epoch_tr_PPV)
    tr_NPV.append(epoch_tr_NPV)
    tr_F1.append(epoch_tr_F1)
    "tr_AUC.append(epoch_tr_AUC)"
    
    te_loss.append(epoch_te_loss)
    te_accuracy.append(epoch_te_accuracy)
    print(epoch_te_accuracy)
    te_MCC.append(epoch_te_MCC)
    te_SE.append(epoch_te_SE)
    te_SPC.append(epoch_te_SPC)
    te_PPV.append(epoch_te_PPV)
    te_NPV.append(epoch_te_NPV)
    te_F1.append(epoch_te_F1)
    "te_AUC.append(epoch_te_AUC)"

In [29]:
te_data = pd.read_csv(r'./batch-size/Word_embedding_TF_valid-8.csv')
te_data_acc = max(te_data.accuracy)
print(f'acc : {te_data_acc}')
print(f'loss : {min(te_data.loss)}')
print(f'mcc : {max(te_data.MCC)}')
print(f'se : {max(te_data.SE)}')
print(f'spc : {max(te_data.SPC)}')
print(f'ppv : {max(te_data.PPV)}')
print(f'npv : {max(te_data.NPV)}')
print(f'f1 : {max(te_data.F1)}')

acc : 0.90625
loss : 0.2586334294546396
mcc : 0.7582582397925111
se : 1.01
spc : 0.99999584891719
ppv : 0.9765166524257988
npv : 0.9699948947637118
f1 : 0.9032649047711644
